In [1]:
import json
import re
from datetime import datetime

In [2]:
with open('../linux-commits-2023-11-12.json') as f:
    all_commits = f.readlines()

In [3]:
all_commits_dict_list = []
all_commits_map = {}
for commit in all_commits:
    commit_dict = json.loads(commit)
    all_commits_map[commit_dict['data']['commit'][0:12]] = commit_dict
    all_commits_dict_list.append(commit_dict)

In [4]:
all_commits_dict_list[-1]

{'backend_name': 'Git',
 'backend_version': '0.13.0',
 'category': 'commit',
 'classified_fields_filtered': None,
 'data': {'Author': 'Linus Torvalds <torvalds@linux-foundation.org>',
  'AuthorDate': 'Sat Nov 11 17:17:22 2023 -0800',
  'Commit': 'Linus Torvalds <torvalds@linux-foundation.org>',
  'CommitDate': 'Sat Nov 11 17:17:22 2023 -0800',
  'Merge': '3ca112b71f35 fd2bd7c0539e',
  'commit': '1b907d0507354b74a4f2c286380cd6059af79248',
  'files': [{'added': '23',
    'file': 'fs/smb/client/cifs_debug.c',
    'removed': '2'},
   {'added': '6', 'file': 'fs/smb/client/cifs_ioctl.h', 'removed': '0'},
   {'added': '2', 'file': 'fs/smb/client/cifsfs.h', 'removed': '2'},
   {'added': '13', 'file': 'fs/smb/client/cifsglob.h', 'removed': '3'},
   {'added': '5', 'file': 'fs/smb/client/cifsproto.h', 'removed': '2'},
   {'added': '40', 'file': 'fs/smb/client/connect.c', 'removed': '8'},
   {'added': '13', 'file': 'fs/smb/client/dfs.c', 'removed': '5'},
   {'added': '1', 'file': 'fs/smb/client/fs

In [5]:
date_format = '%a %b %d %H:%M:%S %Y %z'
def datesDistance(bfc_date, bic_date):
    bfc_date_formated = datetime.strptime(bfc_date, date_format)
    bic_date_formated = datetime.strptime(bic_date, date_format)
    return bfc_date_formated - bic_date_formated

In [6]:
links = []
errors = 0
for commit in all_commits_dict_list:
    try:
        match = re.search("Fixes: (\w*)", commit['data']['message'])
        if match != None:
            bic_hash = match.group(1)
            bic = all_commits_map[bic_hash[0:12]]
            delta = datesDistance(commit['data']['CommitDate'],bic['data']['CommitDate'])
            links.append({
                'BFC_hash': commit['data']['commit'],
                'BIC_hash':  bic['data']['commit'],
                'BFC_comment': commit['data']['message'].split('\n', 1)[0],
                'BIC_comment': bic['data']['message'].split('\n', 1)[0],
                'distance': delta.days,
                'distanceREAL': commit['data']['CommitDate']+"<>"+bic['data']['CommitDate']
            })
    except Exception as e:
        errors+=1
        if 'message' not in commit['data']:
            print(commit['data'])
        else:
            match = re.search("Fixes: (.*)", commit['data']['message'])
            print(match.group(0))
        print(e)

Fixes: http://bugs.gentoo.org/show_bug.cgi?id=87182
'http'
Fixes: http://bugzilla.kernel.org/show_bug.cgi?id=5209
'http'
{'Author': 'Steve French <sfrench@us.ibm.com>', 'AuthorDate': 'Wed Nov 9 15:21:09 2005 -0800', 'Commit': 'Steve French <sfrench@us.ibm.com>', 'CommitDate': 'Wed Nov 9 15:21:09 2005 -0800', 'commit': '7b7abfe3dd81d659a0889f88965168f7eef8c5c6', 'files': [{'action': 'M', 'added': '8', 'file': 'fs/cifs/cifsfs.c', 'indexes': ['877095a1192a', '682b0235ad9a'], 'modes': ['100644', '100644'], 'removed': '3'}], 'parents': ['e82b3aec8d508d2a925a4c766e97f16b7c4dfb1b'], 'refs': []}
'message'
Fixes: IRQ disabled (i915?) when switchig between gnome themes (gnome-theme-manager)
'IRQ'
Fixes: http://bugzilla.kernel.org/show_bug.cgi?id=6706
'http'
Fixes: http://sourceware.org/bugzilla/show_bug.cgi?id=5051
'http'
Fixes: http://bugzilla.kernel.org/show_bug.cgi?id=9825
'http'
Fixes: http://bugzilla.kernel.org/show_bug.cgi?id=9721
'http'
Fixes: http://bugzilla.kernel.org/show_bug.cgi?id=10

In [7]:
len(links)

89059

In [9]:
errors

3606

In [8]:
count = 0
for link in links:
    if link['BIC_hash'] == '1da177e4c3f41524e886b7f1b8a0c1fc7321cac2':
        count+=1
count

456

## Rare cases

-> No commit message: https://github.com/torvalds/linux/commit/7b7abfe3dd81d659a0889f88965168f7eef8c5c6
-> 